# Cihaz Hatalarını Tespit Edecek Tahmin Modeli Geliştiriyoruz

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

### Veri Setini İnceleyelim

In [2]:
df=pd.read_csv("failure.csv")

In [3]:
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        124494 non-null  object
 1   device      124494 non-null  object
 2   failure     124494 non-null  int64 
 3   attribute1  124494 non-null  int64 
 4   attribute2  124494 non-null  int64 
 5   attribute3  124494 non-null  int64 
 6   attribute4  124494 non-null  int64 
 7   attribute5  124494 non-null  int64 
 8   attribute6  124494 non-null  int64 
 9   attribute7  124494 non-null  int64 
 10  attribute8  124494 non-null  int64 
 11  attribute9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [5]:
df.corr(numeric_only=True)

,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
failure,1.000000,0.001984,0.052902,-0.000948,0.067398,0.002270,-0.000550,0.119055,0.119055,0.001622
attribute1,0.001984,1.000000,-0.004248,0.003702,0.001837,-0.003370,-0.001516,0.000151,0.000151,0.001122
attribute2,0.052902,-0.004248,1.000000,-0.002617,0.146593,-0.013999,-0.026350,0.141367,0.141367,-0.002736
attribute3,-0.000948,0.003702,-0.002617,1.000000,0.097452,-0.006696,0.009027,-0.001884,-0.001884,0.532366
attribute4,0.067398,0.001837,0.146593,0.097452,1.000000,-0.009773,0.024870,0.045631,0.045631,0.036069
attribute5,0.002270,-0.003370,-0.013999,-0.006696,-0.009773,1.000000,-0.017051,-0.009384,-0.009384,0.005949
attribute6,-0.000550,-0.001516,-0.026350,0.009027,0.024870,-0.017051,1.000000,-0.012207,-0.012207,0.021152
attribute7,0.119055,0.000151,0.141367,-0.001884,0.045631,-0.009384,-0.012207,1.000000,1.000000,0.006861
attribute8,0.119055,0.000151,0.141367,-0.001884,0.045631,-0.009384,-0.012207,1.000000,1.000000,0.006861
attribute9,0.001622,0.001122,-0.002736,0.532366,0.036069,0.005949,0.021152,0.006861,0.006861,1.000000


In [6]:
# 7 ve 8 aynı alan gibi görünüyor birini silelim
df.drop(["attribute8"], axis=1, inplace=True)

In [7]:
#Hedef alandaki veri dağılımı nedir?
df["failure"].value_counts()

failure
0    124388
1       106
Name: count, dtype: int64

In [8]:
# Aynı cihaza ait birden fazla veri var mı?
df["device"].value_counts()

device
S1F0FGBQ    304
W1F0SJJ2    304
Z1F0GE1M    304
Z1F0KJDS    304
Z1F0KKN4    304
           ... 
Z1F0LT6P      5
Z1F0LT6Z      5
W1F1CJKT      5
S1F04KSC      4
W1F0WJFT      3
Name: count, Length: 1168, dtype: int64

In [9]:
# Aynı tarihte birden fazla kayıt var mı?
df["date"].value_counts()

date
2015-01-01    1163
2015-01-02    1163
2015-01-03    1163
2015-01-04    1162
2015-01-05    1161
              ... 
2015-10-27      31
2015-10-29      31
2015-10-30      31
2015-10-31      31
2015-11-02      31
Name: count, Length: 304, dtype: int64

In [10]:
# Verileri Excelde incelemek için kaydedelim
# df.to_excel("failure.xlsx", index=False)

### Modellemeye başlayalım

In [11]:
# Bağımlı ve bağımsız alanlarımızı ayıralım
x=df.drop(["date", "device", "failure"], axis=1)
y=df[["failure"]]
x.shape, y.shape

((124494, 8), (124494, 1))

In [12]:
#pip install imblearn

In [13]:
# Verilerdeki dengesizliği giderelim
from imblearn.over_sampling import SMOTE

oversample = SMOTE(random_state=42)
x, y = oversample.fit_resample(x, y)

In [14]:
x.shape, y.shape

((248776, 8), (248776, 1))

In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

g = GaussianNB()
b = BernoulliNB()
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
gb= GradientBoostingClassifier()
kn= KNeighborsClassifier()

def algo_test(x, y):
    modeller=[g, b, l, d, r, gb, kn]
    isimler=["GaussianNB", "BernoulliNB", "LogisticRegression", "DecisionTreeClassifier", 
             "RandomForestClassifier", "GradientBoostingClassifier", "KNeighborsClassifier"]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state = 42)
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
    mdl=[]

    print("Veriler hazır modeller deneniyor")
    for model in modeller:
        print(model, " modeli eğitiliyor!..")
        model=model.fit(x_train,y_train)
        tahmin=model.predict(np.array(x_test) if model==kn else x_test)
        mdl.append(model)
        accuracy.append(accuracy_score(tahmin, y_test))
        precision.append(precision_score(tahmin, y_test))
        recall.append(recall_score(tahmin, y_test))
        f1.append(f1_score(tahmin, y_test))
        print(confusion_matrix(tahmin, y_test))

    print("Eğitim tamamlandı.")
    
    metrics=pd.DataFrame(columns=["Accuracy", "Precision", "Recall", "F1", "Model"], index=isimler)
    metrics["Accuracy"] = accuracy
    metrics["Precision"] = precision  
    metrics["Recall"] = recall
    metrics["F1"] = f1
    metrics["Model"]=mdl

    metrics.sort_values("F1", ascending=False, inplace=True)

    print("En başarılı model: ", metrics.iloc[0].name)
    model=metrics.iloc[0,-1]
    tahmin=model.predict(np.array(x_test) if model==kn else x_test)
    print("Confusion Matrix:")
    print(confusion_matrix(tahmin, y_test))
    print("classification Report:")
    print(classification_report(tahmin, y_test))
    print("Diğer Modeller:")
    
    return metrics.drop("Model", axis=1)

In [16]:
# Tüm modelleri deneyelim
algo_test(x,y)

Veriler hazır modeller deneniyor
GaussianNB()  modeli eğitiliyor!..
[[24628 20649]
 [  124  4355]]
BernoulliNB()  modeli eğitiliyor!..
[[21955  1339]
 [ 2797 23665]]
LogisticRegression()  modeli eğitiliyor!..
[[22530 13051]
 [ 2222 11953]]
DecisionTreeClassifier()  modeli eğitiliyor!..
[[24670    33]
 [   82 24971]]
RandomForestClassifier()  modeli eğitiliyor!..
[[24715    10]
 [   37 24994]]
GradientBoostingClassifier()  modeli eğitiliyor!..
[[23650   808]
 [ 1102 24196]]
KNeighborsClassifier()  modeli eğitiliyor!..
[[19781  2359]
 [ 4971 22645]]
Eğitim tamamlandı.
En başarılı model:  RandomForestClassifier
Confusion Matrix:
[[24715    10]
 [   37 24994]]
classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     24725
           1       1.00      1.00      1.00     25031

    accuracy                           1.00     49756
   macro avg       1.00      1.00      1.00     49756
weighted avg       1.00      1.00      1

,Accuracy,Precision,Recall,F1
RandomForestClassifier,0.999055,0.999600,0.998522,0.999061
DecisionTreeClassifier,0.997689,0.998680,0.996727,0.997703
GradientBoostingClassifier,0.961613,0.967685,0.956439,0.962029
BernoulliNB,0.916874,0.946449,0.894301,0.919636
KNeighborsClassifier,0.852681,0.905655,0.819996,0.860699
LogisticRegression,0.693042,0.478044,0.843245,0.610174
GaussianNB,0.582503,0.174172,0.972315,0.295424


In [17]:
# Hedef alanda veri dengesi nedir?
y["failure"].value_counts()

failure
0    124388
1    124388
Name: count, dtype: int64

In [18]:
# Yukarıdaki en başarılı model ile yeniden bir öğrenme gerçekleştirelim
r=RandomForestClassifier()

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state = 42)

In [20]:
model=r.fit(x_train,y_train)

In [21]:
tahmin=model.predict(x_test)

In [22]:
confusion_matrix(tahmin, y_test)

array([[37147,    23],
       [   59, 37404]], dtype=int64)

In [23]:
print(classification_report(tahmin, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37170
           1       1.00      1.00      1.00     37463

    accuracy                           1.00     74633
   macro avg       1.00      1.00      1.00     74633
weighted avg       1.00      1.00      1.00     74633



In [24]:
# Hata vermiş bir makine verisi deneyelim
denek=[[97170872,2576,0,60,12,462175,0,0]]
model.predict(denek)

array([1], dtype=int64)

In [25]:
# Hata vermemeiş bir makine verisi deneyelim
denek=[[239198208,1140,0,60,12,462175,0,0]]
model.predict(denek)

array([0], dtype=int64)

In [26]:
# Veri dengesi sağlandıktan sonra Hedef alanla diğer alanların korelasyonu nasıl oldu?
df=x
df["failure"]=y["failure"]

In [27]:
df.corr(numeric_only=True)

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute9,failure
attribute1,1.000000,-0.087833,0.008579,-0.030105,0.070927,-0.074834,0.194243,0.058853,0.034093
attribute2,-0.087833,1.000000,-0.013723,0.365223,-0.017119,-0.023879,0.020114,-0.024307,0.247283
attribute3,0.008579,-0.013723,1.000000,0.002965,-0.006611,0.010824,-0.009250,0.446410,-0.020971
attribute4,-0.030105,0.365223,0.002965,1.000000,0.002414,-0.035395,0.050530,0.017280,0.222554
attribute5,0.070927,-0.017119,-0.006611,0.002414,1.000000,-0.005932,-0.017559,-0.003242,0.034978
attribute6,-0.074834,-0.023879,0.010824,-0.035395,-0.005932,1.000000,-0.109083,0.029568,-0.008657
attribute7,0.194243,0.020114,-0.009250,0.050530,-0.017559,-0.109083,1.000000,0.232653,0.224206
attribute9,0.058853,-0.024307,0.446410,0.017280,-0.003242,0.029568,0.232653,1.000000,0.037619
failure,0.034093,0.247283,-0.020971,0.222554,0.034978,-0.008657,0.224206,0.037619,1.000000


## Sonuç

Imbalanced (dengesiz) veri seti ile sınıflandırma modeli örneği geliştirdiğimiz bu modelde verilen özelliklerde cihazın hata verip vermeyeceği tahmin edilmeye çalışıldı.